<a href="https://colab.research.google.com/github/IlyaZutler/GBR-agents/blob/main/GBR_Agents_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import matplotlib.pyplot as plt
!pip install simpy
import simpy
import random



In [26]:
np.log(20), np.random.lognormal(3, 0.5)

(2.995732273553991, 29.32014963064012)

In [83]:
class Agent:
    def __init__(self, env, location, speed):
        self.env = env  # Ссылка на объект окружения симуляции
        self.location = location
        self.speed = speed
        self.state = 'free'
        self.alarm = None
        self.busy_start_time = 0  # Время начала занятости
        self.busy_time = 0  # Общее время в состоянии 'busy'
        self.alarms_handled = 0  # Счетчик обработанных тревог

    def travel_time(self, target):
        return abs(self.location - target) / self.speed

    def handle_alarm(self, alarm):
        self.state = 'busy'
        self.alarm = alarm

        self.busy_start_time = self.env.now  # Запоминаем время начала занятости
        self.alarms_handled += 1  # Увеличиваем счетчик обработанных тревог
        #print(f"Agent {id(self)} starting to handle alarm at location {alarm.location} with complexity {alarm.complexity} at time {self.env.now}")

        # Время на дорогу до тревоги
        travel_to_alarm = self.travel_time(alarm.location)
        yield self.env.timeout(travel_to_alarm)

        # Время на обработку тревоги
        #print(f"Agent {id(self)} arrived at alarm at time {self.env.now}")
        yield self.env.timeout(alarm.complexity)

        # Время на возвращение к месту дислокации
        return_travel_time = self.travel_time(self.location)
        yield self.env.timeout(return_travel_time)

        self.location = self.location  # Возвращаемся к исходной дислокации
        # Агенты остаются занятыми до завершения обработки тревоги и возврата к месту дислокации
        #self.busy_time += (self.env.now - self.busy_start_time)  # Обновляем общее время занятости

        self.busy_time += (travel_to_alarm * 2 + alarm.complexity)
        self.state = 'free'  # Возвращаем агента в состояние 'free'
        self.alarm = None
        #print(f"Agent {id(self)} finished handling alarm at time {self.env.now}")

class Alarm:
    def __init__(self, location, complexity):
        self.location = location
        self.complexity = complexity
        self.arrival_time = None

class Simulation:
    def __init__(self, num_agents, segment_length, agent_speed, alarm_rate, complexity_mean, complexity_sigma, threshold, simulation_duration):
        self.env = simpy.Environment()
        # Равномерное распределение агентов вдоль отрезка
        self.agents = [Agent(self.env, segment_length / num_agents / 2 + i * segment_length / num_agents, agent_speed) for i in range(num_agents)]

        self.segment_length = segment_length
        self.alarm_rate = alarm_rate
        self.complexity_mean = complexity_mean
        self.complexity_sigma = complexity_sigma
        self.threshold = threshold
        self.simulation_duration = simulation_duration
        self.alarms = []
        self.total_alarms = 0
        self.total_response_time = 0
        self.delayed_alarms = 0

    def generate_alarms(self):
        while True:
            yield self.env.timeout(random.expovariate(self.alarm_rate))
            location = random.uniform(0, self.segment_length)
            complexity = np.random.lognormal(self.complexity_mean, self.complexity_sigma)
            alarm = Alarm(location, complexity)
            self.alarms.append(alarm)
            self.total_alarms += 1
            #print(f"Generated alarm at location {location} with complexity {complexity} at time {self.env.now}")
            self.env.process(self.dispatch_agent(alarm))

    def dispatch_agent(self, alarm):
        available_agents = [agent for agent in self.agents if agent.state == 'free']
        if available_agents:
            closest_agent = min(available_agents, key=lambda agent: agent.travel_time(alarm.location))
            closest_agent.start_time = self.env.now
            self.total_response_time += closest_agent.travel_time(alarm.location)
            if closest_agent.travel_time(alarm.location) > self.threshold:
                self.delayed_alarms += 1
            yield self.env.process(closest_agent.handle_alarm(alarm))
        else:
            #print(f"No available agents for alarm at location {alarm.location} at time {self.env.now}")
            yield self.env.timeout(0)  # Ничего не делаем, если нет свободных агентов

    def run(self):
        self.env.process(self.generate_alarms())
        self.env.run(until=self.simulation_duration)

    def report(self):
        print(f"Simulation Duration: {self.simulation_duration}")
        print(f"Total Alarms: {self.total_alarms}")

        print(f"Среднее время на тревоге: {np.exp(complexity_mean + 0.5 * complexity_sigma**2)}")
        print(f"Стандартное отклонение времени на тревоге: {( (np.exp(complexity_sigma**2) - 1) * np.exp(2 * complexity_mean + complexity_sigma**2))**0.5 }")

        print(f"Average Response Time: {self.total_response_time / self.total_alarms if self.total_alarms else 0}")
        print(f"Fraction of Delayed Alarms: {self.delayed_alarms / self.total_alarms if self.total_alarms else 0}")
        for i, agent in enumerate(self.agents):
            busy_time_fraction = agent.busy_time / self.simulation_duration
            print(f"Agent {i} Busy Time Fraction: {busy_time_fraction}")
            print(f"Agent {i} Alarms Handled: {agent.alarms_handled}")

def visualize(simulation, interval=1):
    fig, ax = plt.subplots(figsize=(10, 1))
    ax.set_xlim(0, simulation.segment_length)
    ax.set_ylim(-1, 1)

    while simulation.env.peek() < float('inf'):
        ax.clear()
        ax.set_xlim(0, simulation.segment_length)
        ax.set_ylim(-1, 1)

        for agent in simulation.agents:
            if agent.state == 'free':
                ax.plot(agent.location, 0, 'bo')
            elif agent.state == 'busy':
                if agent.alarm is not None:
                    ax.plot(agent.alarm.location, 0, 'mo')
                ax.plot(agent.location, 0, 'go')

        for alarm in simulation.alarms:
            if alarm.arrival_time is None:
                ax.plot(alarm.location, 0, 'ro')

        plt.pause(interval)
        simulation.env.step()

    plt.show()

# Параметры моделирования
days = 1000
num_agents = 5
segment_length = 80   # протяженность в километрах
agent_speed = 30/60  # Cкорость 30 км за 60 мин
alarm_rate = 50/(24*60)  # 50 в день
complexity_mean = np.log(12)  # Cложность 20 мин - подбирается
complexity_sigma = 1 # подбирается
threshold = 15
simulation_duration = 24*60*days  # Задаем время моделирования в минутах (24 часа)

# Создание и запуск симуляции
sim = Simulation(num_agents, segment_length, agent_speed, alarm_rate, complexity_mean, complexity_sigma, threshold, simulation_duration)
sim.run()

# Отчёт о результатах
sim.report()

# Визуализация процесса
#visualize(sim, interval=0.1)

Simulation Duration: 1440000
Total Alarms: 49785
Среднее время на тревоге: 19.78465524840154
Стандартное отклонение времени на тревоге: 25.934368990741053
Average Response Time: 14.8730086795012
Fraction of Delayed Alarms: 0.2777543436778146
Agent 0 Busy Time Fraction: 0.29974117095951053
Agent 0 Alarms Handled: 9266
Agent 1 Busy Time Fraction: 0.3646090915514948
Agent 1 Alarms Handled: 10255
Agent 2 Busy Time Fraction: 0.3767968150214439
Agent 2 Alarms Handled: 10373
Agent 3 Busy Time Fraction: 0.36266343909703963
Agent 3 Alarms Handled: 10154
Agent 4 Busy Time Fraction: 0.3007264823699026
Agent 4 Alarms Handled: 9213


In [79]:
import simpy

def car(env):
    while True:
        print(f"Start parking at {env.now}")
        parking_duration = 5
        yield env.timeout(parking_duration)

        print(f"Start driving at {env.now}")
        trip_duration = 2
        yield env.timeout(trip_duration)

env = simpy.Environment()
env.process(car(env))
env.run(until=15)


Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14
